In [6]:
import pandas as pd
import urllib.request
import os
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import json
import yaml
from calendar import monthrange
import xarray as xr
import cdsapi
import zipfile
from glob import glob

#### IMPORTANT ############
#before running this script, go to cads_forms_json, git pull, then git checkout prod
###########################

